<a href="https://colab.research.google.com/github/kr7/DCNN/blob/main/ITSC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook is an implementation of the paper titled *Classification of Sparse and Irregularly Sampled Time Series with Convolutional Neural Networks*.

Please see the paper for details on the methodology.

In [ ]:
import numpy as np 
import random
import scipy
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

from numpy import genfromtxt
from sklearn.model_selection import  StratifiedKFold

In [ ]:
device = "cuda" # may be set to "cpu" if calculations are not expected to be performed on GPU
file_name_prefix = "Univariate_arff/FaceAll/FaceAll" # set dataset here
SPARSITY = 0.8 # SPARSITY=0.8 means that 80% of the values of the time series will be replaced by zeros (i.e., missing values)
backbone = "FCN" # may be "FCN" or "ResNet"

In [ ]:
!wget http://www.timeseriesclassification.com/Downloads/Archives/Univariate2018_arff.zip

In [ ]:
!unzip Univariate2018_arff.zip

In [ ]:
def sparsify_time_series(time_series, p):
  number_of_missing = int(p*len(time_series))
  for i in range(number_of_missing):
    j = random.randint(0,len(time_series)-1)
    while time_series[j] == 0:
      j = random.randint(0,len(time_series)-1)
    time_series[j] = 0
  return time_series

def sparsify_time_series_dataset(ts_data, p):
  for i in range(len(ts_data)):
    ts_data[i] = sparsify_time_series(ts_data[i], p)
  return ts_data

In [ ]:
# Load train and test data

# Here, we merge the train and test sets, because we will perform 
# 10-fold-crossvalidation later, and we will split the data accordingly

train_data_with_labels = np.genfromtxt(file_name_prefix+"_TRAIN.txt")
test_data_with_labels = np.genfromtxt(file_name_prefix+"_TEST.txt")

data_with_labels = np.vstack( (train_data_with_labels, test_data_with_labels))
data = data_with_labels[:,1:]
data = sparsify_time_series_dataset(data, SPARSITY)
labels = data_with_labels[:,0]

# We make sure that labels are numbered as 0, 1, 2, ... 
# and set the number of classes

min_label = min(labels)
max_label = max(labels)
if min_label == 0:
  NUM_CLASSES = int(max_label+1)
elif min_label == 1:
  labels = labels - min_label
  NUM_CLASSES = int(max_label)
elif min_label == -1:
  if np.sum(labels == -1)+np.sum(labels==1) == len(labels):
    NUM_CLASSES = 2
    labels[labels==-1]=0
  else:
    raise Exception("Unexpected labels")
else:
  raise Exception("Unexpected labels")

NUM_INPUT_FEATURES = len(data[0]) 

In [ ]:
# called "sparsity-aware convolution" or SiConv in the paper
class SparseConvolution(nn.Module): 
    def __init__(self, in_channels, out_channels, kernel_size, stride, padding, device):
        super(SparseConvolution, self).__init__()
        self.stride = stride
        self.padding = padding
        self.conv = nn.Conv1d(in_channels=in_channels, out_channels=out_channels, 
                              kernel_size=kernel_size, stride=stride, padding=padding)
        self.ones = torch.ones(out_channels, in_channels, kernel_size)
        self.ones = self.ones.to(device)

    def forward(self, x):
      non_zero_input = (x!=0).float()
      number_of_non_zero_inputs = F.conv1d(non_zero_input, self.ones, stride=self.stride, padding=self.padding)
      mask = (number_of_non_zero_inputs!=0)
      
      x = self.conv(x)
      x[mask] = x[mask]/number_of_non_zero_inputs[mask]
      return x

In [ ]:
def convolution(convolution_type, in_channels, out_channels, kernel_size, padding="same", device="cpu"):
    if convolution_type == "conventional":
        return nn.Conv1d(in_channels=in_channels, out_channels=out_channels, 
                         kernel_size=kernel_size, padding=padding, stride=1)
    elif convolution_type == "sparse":
        return SparseConvolution(in_channels=in_channels, out_channels=out_channels, 
                         kernel_size=kernel_size, padding=padding, stride=1, device=device)
    else: 
      raise Exception(f"Unknown type of convolution: {convolution_type}")


# FCN = fully-convolutional neural network
class FCN(nn.Module): 
    def __init__(self, convolution_type="conventional", device="cpu"):
        super(FCN, self).__init__()
        
        self.conv1 = convolution(convolution_type, 1, 128, 8, 0, device=device)
        self.bn1 = nn.BatchNorm1d(128)
        self.conv2 = convolution(convolution_type, 128, 256, 5, 0, device=device)
        self.bn2 = nn.BatchNorm1d(256)
        self.conv3 = convolution(convolution_type, 256, 128, 3, 0, device=device)
        self.bn3 = nn.BatchNorm1d(128)

        self.fc = nn.Linear(128*(NUM_INPUT_FEATURES - 13), 128) 
        self.out = nn.Linear(128, NUM_CLASSES) 
        self.softmax = torch.nn.Softmax(dim=1)

        self.to(device)
 

    def forward(self, x):
        x = x.view(-1, 1, NUM_INPUT_FEATURES)
        
        x = self.conv1(x)
        x = self.bn1(x)
        x = torch.relu(x)

        x = self.conv2(x)
        x = self.bn2(x)
        x = torch.relu(x)

        x = self.conv3(x)
        x = self.bn3(x)
        x = torch.relu(x)

        x = x.view(-1, 128*(NUM_INPUT_FEATURES - 13))
        x = self.fc(x)
        x = self.out(x)
        x = self.softmax(x)

        return x
    

# Use this class for experiments with FCN having DConv in the first 
# convolutional layer. For efficient implementation, the activations of the 
# DConv layer will be pre-calculated for all the instances outside this class
# with the dc_activations() function.  
class DFCN(nn.Module): # use this class for experiments with FCN 
    def __init__(self, convolution_type="conventional", device="cpu"):
        super(DFCN, self).__init__()
    
        self.bn1 = nn.BatchNorm1d(256)
        self.conv2 = convolution(convolution_type, 256, 256, 5, 0, device=device)
        self.bn2 = nn.BatchNorm1d(256)
        self.conv3 = convolution(convolution_type, 256, 128, 3, 0, device=device)
        self.bn3 = nn.BatchNorm1d(128)

        self.fc = nn.Linear(128*(NUM_INPUT_FEATURES - 13), 128) 
        self.out = nn.Linear(128, NUM_CLASSES) 
        self.softmax = torch.nn.Softmax(dim=1)

        self.to(device)
 

    def forward(self, x):
        x = x.view(-1, 2*128, NUM_INPUT_FEATURES-8+1)
        
        x = self.bn1(x)
        x = torch.relu(x)

        x = self.conv2(x)
        x = self.bn2(x)
        x = torch.relu(x)

        x = self.conv3(x)
        x = self.bn3(x)
        x = torch.relu(x)

        x = x.view(-1, 128*(NUM_INPUT_FEATURES - 13))
        x = self.fc(x)
        x = self.out(x)
        x = self.softmax(x)

        return x


# use this class for experiments with ResNet with conventional convolution and 
# sparsity-invariant convolution
class ResNet(nn.Module): 
    def __init__(self, convolution_type="conventional", device="cpu"):
        super(ResNet, self).__init__()

        self.conv11 = convolution(convolution_type, 1, 64, 8, device=device) 
        self.bn11 = nn.BatchNorm1d(64)
        self.conv12 = convolution(convolution_type, 64, 64, 5, device=device)
        self.bn12 = nn.BatchNorm1d(64)
        self.conv13 = convolution(convolution_type, 64, 64, 3, device=device) 
        self.bn13 = nn.BatchNorm1d(64)

        self.conv21 = convolution(convolution_type, 64, 128, 8, device=device) 
        self.bn21 = nn.BatchNorm1d(128)
        self.conv22 = convolution(convolution_type, 128, 128, 5, device=device) 
        self.bn22 = nn.BatchNorm1d(128)
        self.conv23 = convolution(convolution_type, 128, 128, 3, device=device) 
        self.bn23 = nn.BatchNorm1d(128)

        self.conv31 = convolution(convolution_type, 128, 128, 8, device=device) 
        self.bn31 = nn.BatchNorm1d(128)
        self.conv32 = convolution(convolution_type, 128, 128, 5, device=device) 
        self.bn32 = nn.BatchNorm1d(128)
        self.conv33 = convolution(convolution_type, 128, 128, 3, device=device) 
        self.bn33 = nn.BatchNorm1d(128)
        
        self.fc = nn.Linear(128*NUM_INPUT_FEATURES, 128) 
        self.out = nn.Linear(128, NUM_CLASSES) 
        self.softmax = torch.nn.Softmax(dim=1)

        self.to(device)
 

    def forward(self, x):
        x = x.view(-1, 1, NUM_INPUT_FEATURES)
        
        x1 = x
        x = self.conv11(x)
        x = self.bn11(x)
        x = torch.relu(x)
        x = self.conv12(x)
        x = self.bn12(x)
        x = torch.relu(x)
        x = self.conv13(x)
        x = self.bn13(x)
        x = torch.relu(x)
        x = x + x1
        x = torch.relu(x)

        x1 = x
        x = self.conv21(x)
        x = self.bn21(x)
        x = torch.relu(x)
        x = self.conv22(x)
        x = self.bn22(x)
        x = torch.relu(x)
        x = self.conv23(x)
        x = self.bn23(x)
        x = torch.relu(x)
        
        x1 = torch.cat( (x1,x1), 1)
        x = x + x1
        x = torch.relu(x)

        x1 = x
        x = self.conv31(x)
        x = self.bn31(x)
        x = torch.relu(x)
        x = self.conv32(x)
        x = self.bn32(x)
        x = torch.relu(x)
        x = self.conv33(x)
        x = self.bn33(x)
        x = torch.relu(x)
        x = x + x1
        x = torch.relu(x)

        x = x.view(-1, 128*(NUM_INPUT_FEATURES))
        x = self.fc(x)
        x = self.out(x)
        x = self.softmax(x)
        
        return x


# Use this class for experiments with ResNet having DConv in the first 
# convolutional layer. For efficient implementation, the activations of the 
# DConv layer will be pre-calculated for all the instances outside this class
# with the dc_activations_resnet() function.  
class DResNet(nn.Module):  
    def __init__(self, convolution_type="conventional", device="cpu"):
        super(DResNet, self).__init__()

        self.bn11 = nn.BatchNorm1d(128)
        self.conv12 = convolution(convolution_type, 128, 64, 5, device=device)
        self.bn12 = nn.BatchNorm1d(64)
        self.conv13 = convolution(convolution_type, 64, 64, 3, device=device) 
        self.bn13 = nn.BatchNorm1d(64)

        self.conv21 = convolution(convolution_type, 64, 128, 8, device=device) 
        self.bn21 = nn.BatchNorm1d(128)
        self.conv22 = convolution(convolution_type, 128, 128, 5, device=device) 
        self.bn22 = nn.BatchNorm1d(128)
        self.conv23 = convolution(convolution_type, 128, 128, 3, device=device) 
        self.bn23 = nn.BatchNorm1d(128)

        self.conv31 = convolution(convolution_type, 128, 128, 8, device=device) 
        self.bn31 = nn.BatchNorm1d(128)
        self.conv32 = convolution(convolution_type, 128, 128, 5, device=device) 
        self.bn32 = nn.BatchNorm1d(128)
        self.conv33 = convolution(convolution_type, 128, 128, 3, device=device) 
        self.bn33 = nn.BatchNorm1d(128)
        
        self.fc = nn.Linear(128*NUM_INPUT_FEATURES, 128) 
        self.out = nn.Linear(128, NUM_CLASSES) 
        self.softmax = torch.nn.Softmax(dim=1) 

        self.to(device)
 

    def forward(self, x):

        x = x.view(-1, 1+2*64, NUM_INPUT_FEATURES)

        x1 = x[:,0,:] # input time series
        x1 = x1.view(-1, 1, NUM_INPUT_FEATURES)

        x_dconv = x[:,1:,:]

        x = self.bn11(x_dconv)
        x = torch.relu(x)
        x = self.conv12(x)
        x = self.bn12(x)
        x = torch.relu(x)
        x = self.conv13(x)
        x = self.bn13(x)
        x = torch.relu(x)
        x = x + x1
        x = torch.relu(x)

        x1 = x
        x = self.conv21(x)
        x = self.bn21(x)
        x = torch.relu(x)
        x = self.conv22(x)
        x = self.bn22(x)
        x = torch.relu(x)
        x = self.conv23(x)
        x = self.bn23(x)
        x = torch.relu(x)
        
        x1 = torch.cat( (x1,x1), 1)
        x = x + x1
        x = torch.relu(x)

        x1 = x
        x = self.conv31(x)
        x = self.bn31(x)
        x = torch.relu(x)
        x = self.conv32(x)
        x = self.bn32(x)
        x = torch.relu(x)
        x = self.conv33(x)
        x = self.bn33(x)
        x = torch.relu(x)
        x = x + x1
        x = torch.relu(x)

        x = x.view(-1, 128*(NUM_INPUT_FEATURES))
        x = self.fc(x)
        x = self.out(x)
        x = self.softmax(x)

        return x

In [ ]:
def eval_net(net, test_data, test_labels, device):
    test_dataset = torch.utils.data.TensorDataset( 
        torch.Tensor(test_data), 
        torch.LongTensor(test_labels)
    )
    testloader = torch.utils.data.DataLoader(test_dataset)

    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, targets in testloader:
            inputs = inputs.to(device) 
            targets = targets.to(device) 
            outputs = net(inputs)
            _, predicted = torch.max(outputs.data, 1)

            total += targets.size(0)
            correct += (predicted == targets)

    return float(correct/total), float(correct), float(total)

In [ ]:
%load_ext cython

In [ ]:
%%cython

import numpy as np
cimport numpy as np

def dtw(np.ndarray[np.float_t, ndim=1] ts1, 
        np.ndarray[np.float_t, ndim=1] ts2):
  
  cdef int LEN_TS1 
  cdef int LEN_TS2
  cdef int i
  cdef int j
  cdef np.ndarray[np.float_t, ndim=2] dtw_matrix

  LEN_TS1 = len(ts1)
  LEN_TS2 = len(ts2)
  
  dtw_matrix = np.zeros( (LEN_TS1, LEN_TS2), dtype=float )
  dtw_matrix[0,0] = abs(ts1[0]-ts2[0])
  
  for i in range(1, LEN_TS1):
      dtw_matrix[i,0] = dtw_matrix[i-1,0]+abs(ts1[i]-ts2[0])

  for j in range(1, LEN_TS2):
      dtw_matrix[0,j] = dtw_matrix[0,j-1]+abs(ts1[0]-ts2[j])

  for i in range(1, LEN_TS1):
      for j in range(1, LEN_TS2):
          dtw_matrix[i,j] = min(dtw_matrix[i-1,j-1], dtw_matrix[i-1,j], 
                          dtw_matrix[i, j-1]) + abs(ts1[i]-ts2[j])
          
  return dtw_matrix[ len(ts1)-1, len(ts2)-1 ]

In [ ]:
def linear_interpolation(ts):
    previous_value = 0
    previous_index = -1
    for i in range(len(ts)):
        if ts[i] != 0:
            previous_index = i
            previous_value = ts[previous_index]
        else:
            next_value = 0
            next_index = i + 1
            while next_index < len(ts) and ts[next_index] == 0:
                next_index = next_index+1
            if next_index < len(ts):
                next_value = ts[next_index]

            w_next = (i-previous_index) / (next_index - previous_index)
            w_previous = 1-w_next

            ts[i] = w_next*next_value + w_previous*previous_value
    return ts


def linear_interpolation_all(ts_data):
    interpolated_data = np.array(ts_data)
    for i in range(len(interpolated_data)):
        interpolated_data[i] = linear_interpolation(interpolated_data[i])
    return interpolated_data


def prune_missing(ts):
    return ts[ts!=0]


def dc_activations(data, convolutional_filters):
    """
    Calculation of the activations of the DConv layer in case of FCN.
    """
    num_instances = len(data)
    length_of_time_series = len(data[0])
    num_conv_filters = len(convolutional_filters)
    conv_filter_size = len(convolutional_filters[0][0])

    activations = np.zeros( (num_instances, 2*num_conv_filters, 
                           length_of_time_series-conv_filter_size+1) )
    for i in range(num_instances):
        for j in range(length_of_time_series-conv_filter_size+1):
            input_segment = data[i,j:j+conv_filter_size]
            input_segment = prune_missing(input_segment)
            if len(input_segment) > 0:
                for k in range(num_conv_filters):
                    activations[i,k,j] = 1.0/(1.0 + dtw(convolutional_filters[k][0], input_segment))
            else:
                for k in range(num_conv_filters):
                    activations[i,k,j] = 0
                    
    for i in range(num_instances):
        for j in range(length_of_time_series-conv_filter_size+1):
            for k in range(num_conv_filters):
                activations[i,num_conv_filters+k,j] = np.sum(convolutional_filters[k][0]*data[i,j:j+conv_filter_size])

    return activations


def dc_activations_resnet(data, convolutional_filters):
    """
    Calculation of the activations of the DConv layer in case of ResNet.
    """
    num_instances = len(data)
    length_of_time_series = len(data[0])
    num_conv_filters = len(convolutional_filters)
    conv_filter_size = len(convolutional_filters[0][0])

    activations = np.zeros( (num_instances, 1+2*num_conv_filters, 
                             length_of_time_series) )
    

    # Because of the residual connections in ResNet, the original data needs to 
    # be returned, this is realized on the 0-th channel
    for i in range(num_instances):
        activations[i,0,:] = data[i,:]


    # zero padding is performed and the length of the output will remain the 
    # same as the length of the input
    for i in range(num_instances):
        for j in range(length_of_time_series-conv_filter_size+1):
            input_segment = data[i,j:j+conv_filter_size]
            input_segment = prune_missing(input_segment)
            for k in range(num_conv_filters):
                activations[i,1+k,j+int(conv_filter_size/2)] = 1.0/(1.0 + dtw(convolutional_filters[k][0],
                                            input_segment))
    for i in range(num_instances):
        for j in range(length_of_time_series-conv_filter_size+1):
            for k in range(num_conv_filters):
                activations[i,1+num_conv_filters+k,j+int(conv_filter_size/2)] = np.sum(convolutional_filters[k][0]*data[i,j:j+conv_filter_size])
                
    return activations

In [ ]:
epochs = 100

accuracies_backbone = []
accuracies_dconv = []
accuracies_linear = []
accuracies_sparse = []


kf =  StratifiedKFold(n_splits=10, random_state=42, shuffle=True)

fold = 0
for train_index, test_index in kf.split(data, labels):
    fold = fold + 1

    train_data = data[train_index]
    train_target = labels[train_index]
    test_data = data[test_index]
    test_target = labels[test_index]

    print(f"Training {backbone}...")
    
    train_dataset = torch.utils.data.TensorDataset(
      torch.Tensor(train_data), torch.LongTensor(train_target) )
    trainloader = torch.utils.data.DataLoader(
      train_dataset, shuffle=True, batch_size=16)
    
    if backbone == "ResNet":
      model = ResNet(device=device)
    elif backbone == "FCN":
      model = FCN(device=device)
    else:
      raise Exception(f"Unknown backbone: {backbone}")

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=1e-5)

    for epoch in range(epochs):  
        running_loss = 0.0
        running_n = 0
        for input_batch, target_batch in trainloader:
            input_batch = input_batch.to(device) 
            target_batch = target_batch.to(device)

            optimizer.zero_grad()

            prediction_batch = model(input_batch) 

            loss = criterion(prediction_batch, target_batch)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            running_n = running_n + 1

        if epoch % 25 == 0 or epoch == epochs-1:
            print(f"epoch: {epoch:3d} loss: {(running_loss/running_n):4.3f}")

    acc, _, _ = eval_net(model, test_data, test_target, device)
    accuracies_backbone.append(acc)
    print(f"Fold: {fold:2d}, accuracy of {backbone}:  {acc:4.3f}")
    
    # ---
    
    print(f"Training {backbone} with DConv...")  
    
    # The previous training with conventional convolution is the first
    # training phase of the model with DConv - the parameters of the DConv
    # layer are considered to be determined by this first training phase
    params = []
    for p in model.parameters():
        params.append(p)
    
    convolutional_filters = np.array(params[0].cpu().detach().numpy(), 
                                  dtype=float)

    # pre-calculations of the activations of the DConv layer
    if backbone == "ResNet":
      dc_activations_train = dc_activations_resnet(train_data, convolutional_filters)
    elif backbone == "FCN":
      dc_activations_train = dc_activations(train_data, convolutional_filters)
    else:
      raise Exception(f"Unknown backbone: {backbone}")

    train_dataset = torch.utils.data.TensorDataset(
      torch.Tensor(dc_activations_train), torch.LongTensor(train_target) )
    trainloader = torch.utils.data.DataLoader(
      train_dataset, shuffle=True, batch_size=16)
    
    if backbone == "ResNet":
      model = DResNet(device=device)
    elif backbone == "FCN":
      model = DFCN(device=device)
    else:
      raise Exception(f"Unknown backbone: {backbone}")

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=1e-5)

    for epoch in range(epochs):
        running_loss = 0.0
        running_n = 0
        for input_batch, target_batch in trainloader:
            input_batch = input_batch.to(device) 
            target_batch = target_batch.to(device)
            optimizer.zero_grad()

            prediction_batch = model(input_batch) 

            loss = criterion(prediction_batch, target_batch)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            running_n = running_n + 1

        if epoch % 25 == 0 or epoch == epochs-1:
            print(f"epoch: {epoch:3d} loss: {(running_loss/running_n):4.3f}")

    if backbone == "ResNet":
      dc_activations_test = dc_activations_resnet(test_data, convolutional_filters)
    if backbone == "FCN":
      dc_activations_test = dc_activations(test_data, convolutional_filters)
    else:
      raise Exception(f"Unknown backbone: {backbone}")
    
    acc, _, _ = eval_net(model, dc_activations_test, test_target, device)
    accuracies_dconv.append(acc)
    print(f"Fold: {fold:2d}, accuracy of {backbone} with DConv:  {acc:4.3f}")
    
    # ---
    
    print(f"Training {backbone} with SiConv...")  
    
    train_dataset = torch.utils.data.TensorDataset(
      torch.Tensor(train_data), torch.LongTensor(train_target) )
    trainloader = torch.utils.data.DataLoader(
      train_dataset, shuffle=True, batch_size=16)

    if backbone == "ResNet":
      model = ResNet("sparse", device=device)
    if backbone == "FCN":
      model = FCN("sparse", device=device)
    else:
      raise Exception(f"Unknown backbone: {backbone}")

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=1e-5)

    for epoch in range(epochs):  
        running_loss = 0.0
        running_n = 0
        for input_batch, target_batch in trainloader:
            input_batch = input_batch.to(device) 
            target_batch = target_batch.to(device)
            optimizer.zero_grad()

            prediction_batch = model(input_batch) 

            loss = criterion(prediction_batch, target_batch)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            running_n = running_n + 1

        if epoch % 25 == 0 or epoch == epochs-1:
            print(f"epoch: {epoch:3d} loss: {(running_loss/running_n):4.3f}")

    acc, _, _ = eval_net(model, test_data, test_target, device)
    accuracies_sparse.append(acc)
    print(f"Fold: {fold:2d}, accuracy of {backbone} with SiConv:  {acc:4.3f}")
      
    # ---
    
    print(f"Training {backbone} with linear interpolation...")
    
    train_dataset = torch.utils.data.TensorDataset(
      torch.Tensor(linear_interpolation_all(train_data)), torch.LongTensor(train_target) )
    trainloader = torch.utils.data.DataLoader(
      train_dataset, shuffle=True, batch_size=16)
    
    if backbone == "ResNet":
      model = ResNet(device=device)
    elif backbone == "FCN":
      model = FCN(device=device)
    else:
      raise Exception(f"Unknown backbone: {backbone}")
    
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=1e-5)

    for epoch in range(epochs):  
        running_loss = 0.0
        running_n = 0
        for input_batch, target_batch in trainloader:
            input_batch = input_batch.to(device) 
            target_batch = target_batch.to(device)
            optimizer.zero_grad()

            prediction_batch = model(input_batch) 

            loss = criterion(prediction_batch, target_batch)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            running_n = running_n + 1

        if epoch % 25 == 0 or epoch == epochs-1:
            print(f"epoch: {epoch:3d} loss: {(running_loss/running_n):4.3f}")

    acc, _, _ = eval_net(model, test_data, test_target, device)
    accuracies_linear.append(acc)
    print(f"Fold: {fold:2d}, accuracy of {backbone} with linear interpolation:  {acc:4.3f}")

    print(accuracies_backbone)
    print(accuracies_linear)
    print(accuracies_sparse)
    print(accuracies_dconv)

In [ ]:
print("**{} (epochs: {})**\n\n".format(file_name_prefix.split('/')[-1], epochs))
print(f"\t\tMean acc. {backbone:6}                           {np.mean(accuracies_backbone):4.3f}")
print(f"\t\tMean acc. {backbone:6} with linear interpolation {np.mean(accuracies_linear):4.3f}")
print(f"\t\tMean acc. {backbone:6} with SiConv               {np.mean(accuracies_sparse):4.3f}")
print(f"\t\tMean acc. {backbone:6} with DConv                {np.mean(accuracies_dconv):4.3f}")
print(f"\t\tStd. acc. {backbone:6}                           {np.std(accuracies_backbone):4.3f}")
print(f"\t\tStd. acc. {backbone:6} with linear interpolation {np.std(accuracies_linear):4.3f}")
print(f"\t\tStd. acc. {backbone:6} with SiConv               {np.std(accuracies_sparse):4.3f}")
print(f"\t\tStd. acc. {backbone:6} with DConv                {np.std(accuracies_dconv):4.3f}")
print(f"\t\tp-value (DConv-backb)                      {scipy.stats.ttest_rel(accuracies_dconv, accuracies_backbone)[1]:4.3f}")
print(f"\t\tp-value (DConv-lin)                        {scipy.stats.ttest_rel(accuracies_dconv, accuracies_linear)[1]:4.3f}")
print(f"\t\tp-value (SiConv-backb)                     {scipy.stats.ttest_rel(accuracies_sparse, accuracies_backbone)[1]:4.3f}")
print(f"\t\tp-value (SiConv-lin)                       {scipy.stats.ttest_rel(accuracies_sparse, accuracies_linear)[1]:4.3f}")
print(f"\t\tp-value (DConv-SiConv)                     {scipy.stats.ttest_rel(accuracies_dconv, accuracies_sparse)[1]:4.3f}")